BRUTE FORCING ALL THE FEATURE FROM 0-62, WITHOUT USING PCA

In [1]:
import pandas as pd
from matplotlib import style
import matplotlib.pyplot as plt
import numpy as np
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [2]:
df = pd.read_csv('/Users/edwardsujono/Python_Project/blog_feed_analytic/data/blogData_train.csv')
df.columns = range(df.shape[1])

In [3]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,271,272,273,274,275,276,277,278,279,280
0,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0
4,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
column_list = range(62)
df_y = df[280].astype(np.int64)
df_x = df[column_list].astype(np.int64)

append the percentile as the label

0-1 -> label_1, 1-4 -> label_2, 4-8 -> label_3, 8-23 -> label_4, 23-100 -> label_5, 100 > -> label_6

Change into class label first

In [5]:
import copy

i = 280

df_categorization = copy.deepcopy(df)

df_categorization.loc[(df_categorization[i] >= 0) & (df_categorization[i] < 1), i] = 0
df_categorization.loc[(df_categorization[i] >= 1) & (df_categorization[i] < 4), i] = 1
df_categorization.loc[(df_categorization[i] >= 4) & (df_categorization[i] < 8), i] = 4
df_categorization.loc[(df_categorization[i] >= 8) & (df_categorization[i] < 23), i] = 8
df_categorization.loc[(df_categorization[i] >= 23) & (df_categorization[i] < 100), i] = 23
df_categorization.loc[df_categorization[i] >= 100, i] = 100

In [6]:
number_cluster = 6
sample_size  = 300
labels = df_categorization[280].as_matrix()
data = df_categorization[range(30)]

In [7]:
def bench_k_means(estimator, name, data, labels):
    t0 = time()
    estimator.fit(data)
    print('init\t\ttime\tinertia\t homo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))


In [8]:

bench_k_means(KMeans(init='random', n_clusters=number_cluster, n_init=number_cluster),
              name="random", data=data, labels=labels)

init		time	inertia	 homo	compl	v-meas	ARI	AMI	silhouette
random   	0.48s	2802728634	0.104	0.071	0.085	0.111	0.071	0.596


DRAW DECISION BOUNDARY

In [ ]:
print ("Starting the PCA")
reduced_data = PCA(n_components=2).fit_transform(data)
print ("Ending the PCA")
kmeans = KMeans(init='k-means++', n_clusters=number_cluster, n_init=number_cluster)
kmeans.fit(reduced_data[:100])

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

Starting the PCA
Ending the PCA
